In [4]:
import pandas as pd
import numpy as np
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [6]:
test.head()

,id,Date,Comment
0,1,20120603163526Z,"""like this if you are a tribe fan"""
1,2,20120531215447Z,"""you're idiot......................."""
2,3,20120823164228Z,"""I am a woman Babs, and the only ""war on women..."
3,4,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F..."
4,5,20120602223825Z,"""haha green me red you now loser whos winning ..."


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
tv.fit(train.Comment)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [15]:
X_train = tv.transform(train.Comment)

In [12]:
len(tv.vocabulary_)

16469

In [13]:
X_test = tv.transform(test.Comment)

In [22]:
from sklearn.svm import SVC
svc = SVC(probability=True)
y_train = train.Insult
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [24]:
from sklearn.model_selection import cross_val_score
cross_val_score(svc, X_train, y_train, scoring='roc_auc')

array([ 0.88664301,  0.86461254,  0.87468188])

In [32]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import roc_auc_score

score_train = []
score_test = []
svc.C = 0.1
cv = ShuffleSplit(X_train.shape[0])
for train_ind, test_ind in cv:
    X_train_cv = X_train[train_ind]
    y_train_cv = y_train[train_ind]
    X_test_cv = X_train[test_ind]
    y_test_cv = y_train[test_ind]
    
    svc.fit(X_train_cv, y_train_cv)
    y_train_pred_cv = svc.predict_proba(X_train_cv)[:, 1]
    y_test_pred_cv = svc.predict_proba(X_test_cv)[:, 1]
    
    score_train.append(roc_auc_score(y_train_cv, y_train_pred_cv))
    score_test.append(roc_auc_score(y_test_cv, y_test_pred_cv))
    
    print(score_train)
    print(score_test)
    print
    

[0.95761972163246034]
[0.89223130841121501]

[0.95761972163246034, 0.95485548165724843]
[0.89223130841121501, 0.93867957870926699]

[0.95761972163246034, 0.95485548165724843, 0.9537994070035698]
[0.89223130841121501, 0.93867957870926699, 0.86926390709870771]



KeyboardInterrupt: 

In [31]:
svc?